# External Data Exploration - Microsoft Malware
So far, there are 4 published Kaggle datasets containing extra information for Microsoft Malware competition. In this kernel, we will explore them:  
* [AvSigVersion Time Stamps][1]
* [Census_OSVersion Time Stamps][2]
* [Google Safe Browsing Report][3]
* [AvSigVersion Threats][4]

These datasets have been uploaded to Kaggle by [Aditya Soni][5], [Chris Deotte][6], [Rob Rose][7], and [Rui Quintino][8]. They have discussion theads [here][9], [here][9], [here][10], and [here][11] respectively. And [here][12], [here][14], and [here][13] are kernels that explore time stamps. Let's load `train.csv`.
  
[1]: https://www.kaggle.com/cdeotte/malware-timestamps
[2]: https://www.kaggle.com/cdeotte/malware-timestamps-2
[3]: https://www.kaggle.com/robroseknows/google-safe-browsing-transparency-report-data/home
[4]: https://www.kaggle.com/rquintino/malware-avsigversion-threats
[5]: https://www.kaggle.com/adityaecdrid
[6]: https://www.kaggle.com/cdeotte
[7]: https://www.kaggle.com/robroseknows
[8]: https://www.kaggle.com/rquintino
[9]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/78672
[10]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/80490
[11]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/81669
[12]: https://www.kaggle.com/cdeotte/time-series-eda-malware-0-64
[13]: https://www.kaggle.com/rquintino/2-months-train-1-month-public-1-day-private
[14]: https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68

In [ ]:
import numpy as np, pandas as pd
load = ['HasDetections', 'AvSigVersion', 'Census_OSVersion', 'OsBuildLab']
df_train = pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load)
df_train['HasDetections'] = df_train['HasDetections'].astype('int8')

# Time Stamps
Version numbers in Microsoft Malware data are associated with time. Microsoft publishes time stamps for `AvSigVersion` and `Census_OSVersion`. Additionally we can deduce time stamps for `OsBuildLab` and `EngineVersion` (EngineVersion is associated with AvSigVersion). ([Here][1] and [here][3] are kernels that explore time stamps. [Here][2] is a kernel that explores time split validation.)  
  
[1]: https://www.kaggle.com/cdeotte/time-series-eda-malware-0-64
[2]: https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68
[3]: https://www.kaggle.com/rquintino/2-months-train-1-month-public-1-day-private

In [ ]:
from datetime import datetime, date, timedelta

# AS timestamp
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy')[()]
df_train['DateAS'] = df_train['AvSigVersion'].map(datedictAS)  

# OS timestamp
datedictOS = np.load('../input/malware-timestamps-2/OSVersionTimestamps.npy')[()]
df_train['DateOS'] = df_train['Census_OSVersion'].map(datedictOS)  

# BL timestamp
def convert(x):
    try:
        d = datetime.strptime(x.split('.')[4],'%y%m%d-%H%M')
    except:
        d = np.nan
    return d
df_train['DateBL'] = df_train['OsBuildLab'].map(convert)
df_train.head()

# Google Safe Browsing Report
[Google][1] publishes information about malware on the internet over time. Let's load this externel dataset, display some rows, merge it with `df_train`, and perform EDA.  
  
[1]: https://transparencyreport.google.com/safe-browsing/overview


In [ ]:
data = pd.read_csv('../input/google-safe-browsing-transparency-report-data/data.csv')
data['WeekOf'] = data['WeekOf'].map(lambda x: datetime.strptime(x,'%Y-%m-%d').date())
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy')[()]
weekdictAS={}
for x in datedictAS: 
    weekdictAS[x] = (datedictAS[x] - timedelta(days= -7+1+datedictAS[x].weekday())).date()
df_train['WeekOf'] = df_train['AvSigVersion'].map(weekdictAS)
df_train = pd.merge(df_train, data, on='WeekOf', how='left')
print('GOOGLE DATA')
data.sample(5)

In [ ]:
import math, calendar
import matplotlib.pyplot as plt
from datetime import datetime

# PARAMETERS
# data : pandas.DataFrame : your data to plot
# col  : str : which column to plot histogram for left y-axis
# target : str : which column for mean/rate on right y-axis
# bars : int : how many histogram bars to show (or less if you set show or min)
# show : float : stop displaying bars after 100*show% of data is showing
# minn : float : don't display bars containing under 100*minn% of data
# sortby : str : either 'frequency', 'category', or 'rate'
# verbose : int : display text summary 1=yes, 0=no
# top : int : give this many bars nice color (and matches a subsequent dynamicPlot)
# title : str : title of plot
# asc : boolean : sort ascending (for category and rate)
# dropna : boolean : include missing data as a category or not

def staticPlot(data, col, target='HasDetections', bars=10, show=1.0, sortby='frequency'
               , verbose=1, top=5, title='',asc=False, dropna=False, minn=0.0):
    # calcuate density and detection rate
    cv = data[col].value_counts(dropna=dropna)
    cvd = cv.to_dict()
    nm = cv.index.values; lnn = len(nm); lnn2 = lnn
    th = show * len(data)
    th2 = minn * len(data)
    sum = 0; lnn2 = 0
    for x in nm[0:bars]:
        lnn2 += 1
        try: sum += cvd[x]
        except: sum += cv[x]
        if sum>th:
            break
        try:
            if cvd[x]<th2: break
        except:
            if cv[x]<th2: break
    if lnn2<bars: bars = lnn2
    pct = round(100.0*sum/len(data),2)
    lnn = min(lnn,lnn2)
    ratio = [0.0]*lnn; lnn3 = lnn
    if sortby =='frequency': lnn3 = min(lnn3,bars)
    elif sortby=='category': lnn3 = 0
    for i in range(lnn3):
        if target not in data:
            ratio[i] = np.nan
        elif nan_check(nm[i]):
            ratio[i] = data[target][data[col].isna()].mean()
        else:
            ratio[i] = data[target][data[col]==nm[i]].mean()
    try: all = pd.DataFrame( {'category':nm[0:lnn],'frequency':[cvd[x] for x in nm[0:lnn]],'rate':ratio} )
    except: all = pd.DataFrame( {'category':nm[0:lnn],'frequency':[cv[x] for x in nm[0:lnn]],'rate':ratio} )
    if sortby=='rate': 
        all = all.sort_values(sortby, ascending=asc)
    elif sortby=='category':
        try: 
            all['temp'] = all['category'].astype('float')
            all = all.sort_values('temp', ascending=asc)
        except:
            all = all.sort_values('category', ascending=asc)
    if bars<lnn: all = all[0:bars]
    if verbose==1 and target in data:
        print('TRAIN.CSV variable',col,'has',len(nm),'categories')
        print('The',min(bars,lnn),'bars displayed here contain',pct,'% of data.')
        mlnn = data[col].isna().sum()
        print("The data has %.1f %% NA. The plot is sorted by " % (100.0*mlnn/len(data)) + sortby )
    
    # plot density and detection rate
    fig = plt.figure(1,figsize=(15,3))
    ax1 = fig.add_subplot(1,1,1)
    clrs = ['red', 'green', 'blue', 'yellow', 'magenta']
    barss = ax1.bar([str(x) for x in all['category']],[x/float(len(data)) for x in all['frequency']],color=clrs)
    for i in range(len(all)-top):
        barss[top+i].set_color('cyan')
    if target in data:
        ax2 = ax1.twinx()
        if sortby!='category': infected = all['rate'][0:lnn]
        else:
            infected=[]
            for x in all['category']:
                if nan_check(x): infected.append( data[ data[col].isna() ][target].mean() )
                elif cvd[x]!=0: infected.append( data[ data[col]==x ][target].mean() )
                else: infected.append(-1)
        ax2.plot([str(x) for x in all['category']],infected[0:lnn],'k:o')
        #ax2.set_ylim(a,b)
        ax2.spines['left'].set_color('red')
        ax2.set_ylabel('Detection Rate', color='k')
    ax1.spines['left'].set_color('red')
    ax1.yaxis.label.set_color('red')
    ax1.tick_params(axis='y', colors='red')
    ax1.set_ylabel('Category Proportion', color='r')
    if title!='': plt.title(title)
    plt.show()
    if verbose==1 and target not in data:
        print('TEST.CSV variable',col,'has',len(nm),'categories')
        print('The',min(bars,lnn),'bars displayed here contain',pct,'% of the data.')
        mlnn = data[col].isna().sum()
        print("The data has %.1f %% NA. The plot is sorted by " % (100.0*mlnn/len(data)) + sortby )
        
# CHECK FOR NAN
def nan_check(x):
    if isinstance(x,float):
        if math.isnan(x):
            return True
    return False

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns, matplotlib.pyplot as plt
from matplotlib import gridspec

# KERNEL DENSITY BANDWITHS
n = [1000,1000,50000,25000,50,1000,1e6,1e6,5,2]
# DENSITY X RANGES
s = [(1,10000),(0,0),(0,0),(700000,950000),(1,750),(1,10000),(1,2e7),(1,2e7),(0,0),(0,0)]
# REMOVE NAN
df_train = df_train[ df_train['AvSigVersion']!='0.0.0.0' ]
# TEXT FORMATTING
def cc(l):
    for i in range(len(l)): 
        l[i] = '< ('+str(i)+') < '+str(l[i])
    return l
# DISCRETIZING FUNCTION FROM 
# https://www.kaggle.com/guoday/nffm-baseline-0-690-on-lb
def make_bucket(data,num=10):
    data.sort()
    bins=[]
    for i in range(num):
        bins.append(data[int(len(data)*(i+1)//num)-1])
    return bins

ct=0
for col in list(data.columns)[1:]:
    print('###############################################')
    print('###     '+col)
    print('###############################################')
    
    # TIME SERIES PLOT
    plt.figure(figsize=(15,5))
    plt.plot(data['WeekOf'],data[col])
    plt.xlabel('Time')
    plt.ylabel(col)
    plt.title('"'+col+'" versus time')
    plt.show()
    
    # HASDETECTION HISTOGRAM
    bins = make_bucket(df_train[col].copy().values,num=20)
    df_train['P']=np.digitize(df_train[col],bins=bins)
    staticPlot(df_train[ df_train['P']!=20 ],'P',sortby='category',asc=True,bars=20,verbose=0,
               title='HasDetections Rate versus "'+col+'" (Bars use left y-axis. Dotted line uses right)')
    print('KEY TO BINS:',col,cc(bins))
    
    # DENSITY PLOT AND BOXPLOT
    lines = [1, 0]
    plt.figure(figsize=(15,5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
    plt.subplot(gs[0])
    for line in lines:
        subset = df_train[ (df_train['HasDetections'] == line) & (~df_train[col].isna())]
        sns.kdeplot(subset[col], bw=n[ct],
                     shade=True, linewidth=3, 
                     label = line)
    plt.legend(prop={'size': 16}, title = 'HasDetections')
    plt.title('Density Plot of HasDetections versus "'+col+'"')
    plt.xlabel(col)
    if s[ct][0]!=0: plt.xlim((s[ct][0],s[ct][1]))
    plt.ylabel('Density')
    ax = plt.subplot(gs[1])
    df_train2 = df_train[ ~df_train[col].isna() ]
    plt.boxplot([df_train2[ df_train2['HasDetections']==0][col],df_train2[ df_train2['HasDetections']==1][col]])
    plt.title('Boxplot of "'+col+'"')
    if s[ct][0]!=0: plt.ylim((s[ct][0],s[ct][1]))
    ct += 1
    plt.xlabel('HasDetections')
    ax.set_xticklabels([0,1])
    plt.show()
del df_train['P']

# AvSigVersion Threats
[Microsoft][1] publishes malware threats. For each `AvSigVersion`, this external dataset lists all the known malware that was threatening it. (Note that for each point in time, all (95%) of computers use the same `AvSigVersion`, so threats on `AvSigVersion` can be interpreted as threats at a given time point.) There is a lot of information in this dataset, we will only add one new feature to `df_train`. For each `AvSigVersion`, we will count how many things were threatening it, and add a new varable, `ThreatCount`.  
  
[1]: https://www.microsoft.com/en-us/wdsi/definitions/antimalware-definition-release-notes?RequestVersion=1.277.43.0

In [ ]:
data2 = pd.read_csv('../input/malware-avsigversion-threats/AvSigversion_Threats.csv')
cv = pd.DataFrame(data2.groupby('AvSigVersion')['index'].count()).rename({'index':'ThreatCount'},axis=1)
df_train = pd.merge(df_train,cv,on='AvSigVersion',how='left')
df_train['ThreatCount'].fillna(0,inplace=True)
print('THREAT DATA')
data2.sample(10)

In [ ]:
df_train[['AvSigVersion','OsBuildLab','Census_OSVersion','HasDetections','ThreatCount']].sample(5)

In [ ]:
staticPlot(df_train,'ThreatCount',sortby='category',asc=True,bars=10
           ,title='Threat Count, 10 most frequent (65% of all data)(Bars use left y-axis. Dotted line uses right)',verbose=False)
staticPlot(df_train,'ThreatCount',sortby='category',asc=True,bars=100
           ,title='Threat Count, 100 most frequent (90% of all data)(Bars use left y-axis. Dotted line uses right)',verbose=False)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns, matplotlib.pyplot as plt
from matplotlib import gridspec

# DENSITY PLOT AND BOXPLOT
col = 'ThreatCount'
lines = [1, 0]
plt.figure(figsize=(15,5))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
plt.subplot(gs[0])
for line in lines:
    subset = df_train[ (df_train['HasDetections'] == line) & (~df_train[col].isna())]
    sns.kdeplot(subset[col], bw=10,
                   shade=True, linewidth=3, 
                   label = line)
plt.legend(prop={'size': 16}, title = 'HasDetections')
plt.title('Density Plot of HasDetections versus "'+col+'"')
plt.xlabel(col)
plt.xlim((-50,500))
plt.ylabel('Density')
ax = plt.subplot(gs[1])
df_train2 = df_train[ ~df_train[col].isna() ]
plt.boxplot([df_train2[ df_train2['HasDetections']==0][col],df_train2[ df_train2['HasDetections']==1][col]])
plt.title('Boxplot of "'+col+'"')
plt.ylim((-50,500))
plt.xlabel('HasDetections')
ax.set_xticklabels([0,1])
plt.show()

# LGBM Feature Importance
We will now train an LGBM model on just variables from the 2 external datasets, Google data and Threat data. First we will delete all other variables.

In [ ]:
del df_train['DateAS'], df_train['DateOS'], df_train['DateBL'], df_train['WeekOf'] 
del df_train['AvSigVersion'], df_train['OsBuildLab'], df_train['Census_OSVersion']
print('TRAIN DATA')
df_train.sample(5)

In [ ]:
import lightgbm as lgb,gc

# CREATE TRAIN AND VALIDATE
X_train = df_train.sample(frac=0.5)
Y_train = X_train['HasDetections']
X_val = df_train[ ~df_train.index.isin(X_train.index) ]
Y_val = X_val['HasDetections']
del X_train['HasDetections'], X_val['HasDetections'], df_train
x=gc.collect()

# TRAIN LGBM
model = lgb.LGBMClassifier(n_estimators=10000, colsample_bytree=0.7, objective='binary', num_leaves=32,
            max_depth=-1, learning_rate=0.1)
h=model.fit(X_train, Y_train, eval_metric='auc', eval_set=[(X_val, Y_val)], verbose=50,
            early_stopping_rounds=100)

# FEATURE IMPORTANCE
df = pd.DataFrame({"mean" : model.feature_importances_, "feature" : X_train.columns })
df.sort_values("mean", inplace=True)
ax = df.plot(x="feature", y="mean", kind='barh',color='green', figsize=(12, 20)
             , title='External Data LGBM Feature Importance')

# Predict test.csv and submit

In [ ]:
del X_train, X_val, Y_train, Y_val
# LOAD TEST
load = ['AvSigVersion', 'Census_OSVersion', 'OsBuildLab']
df_test = pd.read_csv('../input/microsoft-malware-prediction/test.csv',dtype='category',usecols=load)
# GOOGLE DATA
df_test['WeekOf'] = df_test['AvSigVersion'].map(weekdictAS)
df_test = pd.merge(df_test, data, on='WeekOf', how='left')
# THREAT DATA
df_test = pd.merge(df_test,cv,on='AvSigVersion',how='left')
df_test['ThreatCount'].fillna(0,inplace=True)
# DELETE EXTRA
del df_test['WeekOf']
# DELETE ORIGINAL
del df_test['AvSigVersion'], df_test['OsBuildLab'], df_test['Census_OSVersion']
x=gc.collect()
print('TEST DATA')
df_test.sample(5)

In [ ]:
# PREDICT IN CHUNKS FOR REDUCED MEMORY USAGE
idx = 0; chunk = 2000000
pred_val = np.zeros(len(df_test))
while idx < len(df_test):
    idx2 = min(idx + chunk, len(df_test) )
    idx = range(idx, idx2)
    pred_val[idx] = model.predict_proba(df_test.iloc[idx])[:,1]
    idx = idx2
submit = pd.read_csv('../input/microsoft-malware-prediction/sample_submission.csv')
submit['HasDetections'] = pred_val
submit.to_csv('ExternalData.csv', index=False)

![image](http://playagricola.com/Kaggle/sub22819.png)

# Conclusion
In this kernel, we explored data from the 4 external Microsoft malware datasets. We attempted to predict Malware HasDetections from just the data contained therein but were not successful. We only achieved an LB of 0.500. The data in these datasets looks useful, so if we try other encodings, maybe we can gain some predictive power. I encorage others to fork this kernel and explore. If you make any discoveries or achieve a submission over LB 0.500, please share your work in the comments below.